Modified from https://colab.research.google.com/github/facebookresearch/esm/blob/master/examples/sup_variant_prediction.ipynb

<a id='prelims'></a>
## Prelims

If you are using colab, then uncomment and run the cell below.
It will pip install the `esm` code, fetch the fasta file and the pre-computed embeddings.

In [ ]:
!pip install git+https://github.com/facebookresearch/esm.git

!curl -LO https://github.com/tcoard/Deep_Learning_For_Proteins/raw/master/coala40_reprs.tar.gz
!tar -xzf coala40_reprs.tar.gz 
!curl -LO https://github.com/tcoard/Deep_Learning_For_Proteins/raw/master/prot_bert_embeddings.tar.gz
!tar -xzf prot_bert_embeddings.tar.gz
!curl -LO https://transfer.sh/tq5EsK/model.h5

!curl -LO https://github.com/tcoard/Deep_Learning_For_Proteins/raw/master/reformatted_coala40.fa

!pwd
!ls

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/esm.git to /tmp/pip-req-build-wqjqm33_
  Running command git clone -q https://github.com/facebookresearch/esm.git /tmp/pip-req-build-wqjqm33_
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for fair-esm: filename=fair_esm-0.5.0-py3-none-any.whl size=69640 sha256=a27f4e209d021a3160644716e4aef5e50d97a01af8115c5fcec52987595e0d92
  Stored in directory: /tmp/pip-ephem-wheel-cache-i23vqupm/wheels/5c/8e/11/307eca5379b418a9989ef4d24ead655755f2500a577ec95208
Successfully built fair-esm
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 23.5M  100 23.5M    0     0  25.1M    

The embeddings were generated with:

```python esm/extract.py esm1_t34_670M_UR50S reformatted_coala40.fa coala40_reprs/ --repr_layers 34 --include mean```

## Import Statements

In [ ]:
import random
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import esm
import scipy

from collections import Counter
from tqdm import tqdm
from tensorflow.keras import models, layers, regularizers, optimizers
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.svm import SVC, SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report

## Embedding Configuration
COALA Dataset: https://www.biorxiv.org/content/10.1101/2020.04.17.047316v1.abstract

ESM (source paper): https://www.pnas.org/doi/10.1073/pnas.2016239118

PROTBERT: https://www.biorxiv.org/content/10.1101/2020.07.12.199554v3


---




In [ ]:
FASTA_PATH = "./reformatted_coala40.fa" #@param
EMBEDDING_METHOD = "ESM" #@param ["ESM", "PROTBERT"]
if EMBEDDING_METHOD == "ESM":
    EMB_PATH = "./coala40_reprs/"
elif EMBEDDING_METHOD == "PROTBERT":
    EMB_PATH = "./prot_bert_embeddings"
EMB_LAYER = 34

<a id='load_embeddings'></a>
## Load embeddings (Xs) and target effects (ys)
Our FASTA file is formatted as such:
```
>{index}|{mutation_id}|{effect}
{seq}
```
We will be extracting the effect from each entry.

Our embeddings are stored with the file name from fasta header: `{index}|{mutation_id}|{effect}.pt`. 
For example,

In [ ]:
#@title What the FASTA Looks like:
!cat reformatted_coala40.fa | head -10

In [ ]:
AA_LIST = ('X', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 
           'Q', 'R', 'S', 'T', 'V', 'W', 'Y')
# tokenizes a list of sequences into numerical forms (numbers from 0 to 20)
def tokenize_sequences(sequences, SeqCol='ISM', seqlen=600):
    # limits the length of sequences to a fixed value
    def fix_size(x):
        if len(x) < seqlen:
            return x + 'X'*(seqlen-len(x))
        elif len(x) > seqlen:
            return x[:seqlen]
        else:
            return x

    # fix length of each sequence, and place them into a numpy array
    data = np.vstack([list(fix_size(sequence)) for sequence in sequences])

    # tokenize (i.e. assign a numerical value) each amino acid
    aa_tokenizer = {AA_LIST[k]:k for k in range(len(AA_LIST))}

    # apply this to the entire data
    return np.vectorize(aa_tokenizer.get)(data)

# ***************************************************
labels = [] # the drug resistances
sequences = [] # the raw sequences
raw_embeddings = [] # the raw (pretrained) embeddings
# ***************************************************

for header, _seq in esm.data.read_fasta(FASTA_PATH):
    sequences.append(_seq)

    scaled_effect = header.split('|')[-1]
    labels.append(scaled_effect)

    # We will load premade embeddings for each protein by their header
    embs = torch.load(f'{EMB_PATH}/{header[1:]}.pt')
    
    if EMBEDDING_METHOD == "ESM":
        raw_embeddings.append(embs['mean_representations'][EMB_LAYER])
    elif EMBEDDING_METHOD == "PROTBERT":
        raw_embeddings.append(embs)

# Reform the list of embeddings into a numpy array of all embeddings
# (was a list of tensors previously)
embeddings = torch.stack(raw_embeddings, dim=0).numpy()
sequence_OH = tokenize_sequences(sequences)

# Part 1: End-to-End Model

In [ ]:
#@title Example of Tokenized Sequences
print(sequences[0])
print(sequence_OH[0])

In [ ]:
#@title Sequence Length Graph
import matplotlib.pyplot as plt

# plt.xlim([0, 1800])
sequenceLengths = [len(_seq) for _seq in sequences]
plt.hist(sequenceLengths, alpha=0.5)
plt.title('Sequence Lengths')
plt.xlabel('Length')
plt.ylabel('count')
plt.yscale("log")

plt.show()
# sequenceLengths

### Train / Test Split

The Envision paper this notebook was based on used 80% of the data for training, but it was found that pre-trained ESM embeddings require fewer downstream training data to reach comprable performance.

In [ ]:
#@title Training Parameters
train_size = 0.8 #@param {type:"slider", min:0, max:1, step:0.05}
X_embed_train, X_embed_test, y_train, y_test = train_test_split(embeddings, labels, train_size=train_size, random_state=3351)
X_OH_train, X_OH_test, y_train, y_test = train_test_split(sequence_OH, labels, train_size=train_size, random_state=3351)

We'll do some processing on the data to make them more compatable with our networks.

OneHotEncoder: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [ ]:
labelEncoder = OneHotEncoder()
y_train_OH = labelEncoder.fit_transform(
    np.array(y_train).reshape(-1, 1)
).toarray()

drug_index = {
    resistance: i
    for i, resistance in enumerate(list(labelEncoder.categories_[0]))
}

y_train_numerical = np.array([drug_index[drug] for drug in y_train])
y_test_numerical = np.array([drug_index[drug] for drug in y_test])

In [ ]:
#@title One-Hot Encoding (for training)
OH = labelEncoder.fit_transform(labelEncoder.categories_[0].reshape(-1, 1)).todense()
for row, resistance in zip(OH, labelEncoder.categories_[0]):
  print(f"{resistance:>27}: {row}")

In [ ]:
#@title Numerical Encoding (for direct comparison)
for key, value in drug_index.items():
  print(f"{key:>27}: {value}")

In [ ]:
#@title Class Distribution
labels, counts = np.unique(y_train, return_counts = True)
dict(zip(labels, counts))

##Model
For the default parameters, it is recommended to run for about 250 epochs. The partially-trained model is 200 epochs in, so 50 more would be advisable. Note that changing the number of dense layer nodes or the activation function will not take effect unless train_from_scratch is checked.

In [ ]:
epochs = 10 #@param {type:"slider", min:10, max:200, step:5}
train_from_scratch = False #@param {type:"boolean"}
dense_layer_nodes = 64 #@param {type:"slider", min:8, max:128, step:1}
activation_function = "relu" #@param ["relu", "tanh", "sigmoid"]

In [ ]:
if train_from_scratch:
    model = models.Sequential()

    # Embedding Layer
    emb_layer = layers.Embedding(input_dim=21, output_dim=10)
    emb_layer._name = "embedding_layer"
    model.add(emb_layer)
    model.add(layers.Dropout(0.25))

    # Convlutional Neural Networks
    model.add(
        layers.Conv1D(
            filters=32,
            kernel_size=15,
            activation=activation_function,
            kernel_regularizer=regularizers.L2(5e-4),
            padding="same",
            bias_regularizer=regularizers.L2(0.0001),
        )
    )
    model.add(layers.Dropout(0.2))
    model.add(layers.MaxPooling1D())

    model.add(
        layers.Conv1D(
            filters=16,
            kernel_size=5,
            activation=activation_function,
            kernel_regularizer=regularizers.L2(8e-4),
            padding="same",
            bias_regularizer=regularizers.L2(0.0001),
        )
    )
    model.add(layers.Dropout(0.2))
    model.add(layers.MaxPooling1D())

    model.add(
        layers.Conv1D(
            filters=8,
            kernel_size=3,
            activation=activation_function,
            kernel_regularizer=regularizers.L2(4e-4),
            padding="same",
            bias_regularizer=regularizers.L2(0.0001),
        )
    )
    model.add(layers.Dropout(0.2))
    model.add(layers.GlobalMaxPooling1D())

    # Dense Layers
    model.add(
        layers.Dense(
            units=dense_layer_nodes,
            activation=activation_function,
            kernel_regularizer=regularizers.L2(0.001),
            bias_regularizer=regularizers.L2(0.0001),
        )
    )
    final_dense = layers.Dense(
        units=dense_layer_nodes,
        activation=activation_function,
        kernel_regularizer=regularizers.L2(0.001),
        bias_regularizer=regularizers.L2(0.0001),
    )
    final_dense._name = "final_dense_layer"
    model.add(final_dense)

    # Output Layer
    model.add(layers.Dense(units=10, activation="softmax"))

    o = optimizers.Adam(learning_rate=5e-3)
    model.compile(
        optimizer=o,
        loss="categorical_crossentropy",
        metrics=["accuracy", "Precision", "Recall"],
    )
else:
    model = models.load_model("model.h5")

history = model.fit(
    X_OH_train,
    y_train_OH,
    validation_split=0.2,
    epochs=epochs,
    batch_size=128,
    use_multiprocessing=True,
    verbose=1,
)

In [ ]:
# @title What Does the Embedding Layer Do?
for index, amino_acid in enumerate(AA_LIST):
  print(f"{amino_acid}: {model.get_layer('embedding_layer').get_weights()[0][index]}")

In [ ]:
#@title Training Accuracy and Loss
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#@title Classification Reports
from sklearn.metrics import classification_report
values = np.argmax(model.predict(X_OH_test), axis = 1)
actual = np.array(y_test_numerical)

print(classification_report(y_true = actual, y_pred = values, target_names = drug_index.keys()))

In [ ]:
from keras.models import Model
embedding_model = Model(inputs = model.input, 
             outputs = model.get_layer("final_dense_layer").output)
x_emb = embedding_model.predict(X_OH_train)

In [ ]:
#@title Graph Embeddings
iterations = 250 #@param {type:"slider", min:250, max:2000, step:50}
tsne = TSNE(init = "pca", perplexity = 15, early_exaggeration=20, n_iter = iterations)
X_embed_train_tsne = tsne.fit_transform(x_emb)
sns.set(rc={"figure.figsize": (10, 10)})  
palette = sns.color_palette("hls", len(set(y_train)))   
sc = sns.scatterplot(X_embed_train_tsne[:,0], X_embed_train_tsne[:,1], hue=y_train ,marker='.', legend="full", s=100, palette=palette)
plt.legend(bbox_to_anchor=(1.005, 1.0), loc=2, borderaxespad=0.0)  

# Part 2: Pretrained Embeddings

### PCA

Principal Component Analysis (PCA) is a dimensionality-reduction method that transforms a large set of variables into a smaller one while still containing 
most of the information in the large set.

It does this through a process of computing the principal components and using them to perform a change of basis on the data. 

Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [ ]:
pca = PCA(n_components=20)
X_embed_train_pca = pca.fit_transform(X_embed_train)
X_embed_test_pca = pca.transform(X_embed_test) # for a later step
print(X_embed_train.shape)
print(X_embed_train_pca.shape)

<a id='viz_embeddings'></a>
## Visualize Embeddings

Here, we plot the first two principal components on the x- and y- axes. Each point is then colored by its scaled effect (what we want to predict).

Visually, we can see a separation based on color/effect, suggesting that our representations are useful for this task, without any task-specific training!

In [ ]:
#@title PCA Plot

%timeit
sns.set(rc={"figure.figsize": (10, 10)})  
palette = sns.color_palette("hls", len(set(y_train)))   
sc = sns.scatterplot(X_embed_train_pca[:,0], X_embed_train_pca[:,1], hue=y_train ,marker='.', legend="full", s=100, palette=palette)
plt.legend(bbox_to_anchor=(1.005, 1.0), loc=2, borderaxespad=0.0)  

### TSNE
t-distributed stochastic neighbor embedding (t-SNE) is a dimensionality-reduction method that transforms a large set of variables into a smaller one by modeling each high-dimensional object by a two- or three-dimensional point in such a way that similar objects are modeled by nearby points and dissimilar objects are modeled by distant points with high probability. You can modify the number of iterations that the t-SNE runs with the slider; consider keeping the value below 500 if you are on CPU.

Interactive and in depth explanation of t-SNE. https://observablehq.com/@robstelling/t-sne_en

Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html

In [ ]:
#@title TSNE Plot
iterations_ = 750 #@param {type:"slider", min:250, max:2000, step:50}
tsne = TSNE(init = "pca", n_iter = iterations_)
X_embed_train_tsne = tsne.fit_transform(X_embed_train)

sns.set(rc={"figure.figsize": (10, 10)})  
palette = sns.color_palette("hls", len(set(y_train)))   
sc = sns.scatterplot(X_embed_train_tsne[:,0], X_embed_train_tsne[:,1], hue=y_train ,marker='.', legend="full", s=100, palette=palette)
plt.legend(bbox_to_anchor=(1.005, 1.0), loc=2, borderaxespad=0.0)  

## Clustering on Embeddings
We will use two different regression models:
1. [K-nearest-neighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)
2. [Random Forest Regressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html?highlight=randomforestregressor#sklearn.ensemble.RandomForestRegressor)

We will be fitting these classifiers on the PCA-transformed embeddings, as they perform just as well as the raw embeddings while also being significantly faster. Don't believe it? Try swapping X-embed_train_pca for X_embed_train (and likewise for test).

In [ ]:
knn = KNeighborsClassifier(
    n_neighbors = 10,
    weights = "uniform",
    algorithm = "kd_tree",
    leaf_size = 15,
    p = 1,
)

rfc = RandomForestClassifier(
    n_estimators = 20,
    criterion = "gini",
    min_samples_split = 10,
    min_samples_leaf = 4,
    max_features = "sqrt"
)

In [ ]:
#@title Classification Reports: KNN
knn.fit(X_embed_train_pca, y_train_numerical)
pred_y = knn.predict(X_embed_test_pca)
print(classification_report(y_true = actual, y_pred = pred_y, target_names = drug_index.keys()))

In [ ]:
#@title Classification Reports: RFC
rfc.fit(X_embed_train_pca, y_train_numerical)
pred_y = rfc.predict(X_embed_test_pca)
print(classification_report(y_true = actual, y_pred = pred_y, target_names = drug_index.keys()))

<a id='grid_search'></a>

## Initialize / Run GridSearch

(Skipped in demo for time reasons)

Finding the right parameters for any classifiers is a challenge. To aid in finding them for KNN and K-Means from above, we ran [grid search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). It works by generating multiple permutations of possible parameters, and testing them with cross-validation.

In [ ]:
knn_grid = {
    'n_neighbors': [5, 10],
    'weights': ['uniform', 'distance'],
    'algorithm': ['ball_tree', 'kd_tree', 'brute'],
    'leaf_size' : [15, 30],
    'p' : [1, 2],
}

# only added with the checkbox active above
rfc_grid = {
    'n_estimators' : [20],
    'criterion' : ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split' : [5, 10],
    'min_samples_leaf': [1, 4]
}

In [ ]:
cls_list = [KNeighborsClassifier, RandomForestClassifier]
param_grid_list = [knn_grid, rfc_grid]

### Run Grid Search 

In [ ]:
result_list = []
grid_list = []

In [ ]:
for cls_name, param_grid in zip(cls_list, param_grid_list):
    print(cls_name)
    grid = GridSearchCV(
        estimator = cls_name(), 
        param_grid = param_grid,
        scoring = 'r2',
        verbose = 1,
        n_jobs = -1 # use all available cores
    )
    grid.fit(X_embed_train_pca, y_train_numerical)
    result_list.append(pd.DataFrame.from_dict(grid.cv_results_))
    grid_list.append(grid)

<class 'sklearn.neighbors._classification.KNeighborsClassifier'>
Fitting 5 folds for each of 48 candidates, totalling 240 fits
<class 'sklearn.ensemble._forest.RandomForestClassifier'>
Fitting 5 folds for each of 16 candidates, totalling 80 fits


<a id='browse'></a>
## Browse the Sweep Results

The following tables show the top 5 parameter settings, based on `mean_test_score`. Given our setup, this should really be thought of as `validation_score`.

K Nearest Neighbors: 
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

In [ ]:
result_list[0].sort_values('rank_test_score')[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_leaf_size,param_n_neighbors,param_p,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
12,0.007336,0.000154,0.166533,0.006463,ball_tree,30,10,1,uniform,"{'algorithm': 'ball_tree', 'leaf_size': 30, 'n...",-0.129543,-0.07142,-0.019877,-0.053704,-0.085688,-0.072047,0.036195,1
20,0.013959,0.002751,0.212152,0.006830,kd_tree,15,10,1,uniform,"{'algorithm': 'kd_tree', 'leaf_size': 15, 'n_n...",-0.129543,-0.07142,-0.019877,-0.053704,-0.085688,-0.072047,0.036195,1
36,0.001604,0.000094,0.223431,0.049854,brute,15,10,1,uniform,"{'algorithm': 'brute', 'leaf_size': 15, 'n_nei...",-0.129543,-0.07142,-0.019877,-0.053704,-0.085688,-0.072047,0.036195,1
4,0.009240,0.001838,0.180414,0.004543,ball_tree,15,10,1,uniform,"{'algorithm': 'ball_tree', 'leaf_size': 15, 'n...",-0.129543,-0.07142,-0.019877,-0.053704,-0.085688,-0.072047,0.036195,1
28,0.013544,0.002711,0.239430,0.042414,kd_tree,30,10,1,uniform,"{'algorithm': 'kd_tree', 'leaf_size': 30, 'n_n...",-0.129543,-0.07142,-0.019877,-0.053704,-0.085688,-0.072047,0.036195,1


### Random Forest

In [ ]:
# Here is how to see the results of the random forest if it was run before
# (requires you to have had RFC on)
result_list[1].sort_values('rank_test_score')[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_leaf_size,param_n_neighbors,param_p,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
12,0.007438,0.000137,0.166322,0.006498,ball_tree,30,10,1,uniform,"{'algorithm': 'ball_tree', 'leaf_size': 30, 'n...",-0.129543,-0.07142,-0.019877,-0.053704,-0.085688,-0.072047,0.036195,1
20,0.011582,0.000237,0.216946,0.009255,kd_tree,15,10,1,uniform,"{'algorithm': 'kd_tree', 'leaf_size': 15, 'n_n...",-0.129543,-0.07142,-0.019877,-0.053704,-0.085688,-0.072047,0.036195,1
36,0.001517,0.000052,0.132545,0.004368,brute,15,10,1,uniform,"{'algorithm': 'brute', 'leaf_size': 15, 'n_nei...",-0.129543,-0.07142,-0.019877,-0.053704,-0.085688,-0.072047,0.036195,1
4,0.008258,0.000186,0.164024,0.003186,ball_tree,15,10,1,uniform,"{'algorithm': 'ball_tree', 'leaf_size': 15, 'n...",-0.129543,-0.07142,-0.019877,-0.053704,-0.085688,-0.072047,0.036195,1
28,0.013625,0.004405,0.180024,0.006990,kd_tree,30,10,1,uniform,"{'algorithm': 'kd_tree', 'leaf_size': 30, 'n_n...",-0.129543,-0.07142,-0.019877,-0.053704,-0.085688,-0.072047,0.036195,1
